In [12]:
# prompt: connect google

from google.colab import auth
auth.authenticate_user()
print('Authenticated')



Authenticated


In [13]:
path ='/content/drive/MyDrive/EVENT/screen_time.csv'

In [14]:
# prompt: read path and print content

from google.colab import drive
drive.mount('/content/drive')

path ='/content/drive/MyDrive/EVENT/screen_time.csv'

try:
  with open(path, 'r') as file:
    content = file.read()
    print(content)
except FileNotFoundError:
  print(f"File not found at {path}")
except Exception as e:
  print(f"An error occurred: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Age,Gender,Screen Time Type,Day Type,Average Screen Time (hours),Sample Size
5,Male,Educational,Weekday,0.44,500
5,Male,Recreational,Weekday,1.11,500
5,Male,Total,Weekday,1.55,500
5,Male,Educational,Weekend,0.5,500
5,Male,Recreational,Weekend,1.44,500
5,Male,Total,Weekend,1.93,500
5,Female,Educational,Weekday,0.49,500
5,Female,Recreational,Weekday,0.96,500
5,Female,Total,Weekday,1.45,500
5,Female,Educational,Weekend,0.5,500
5,Female,Recreational,Weekend,1.4,500
5,Female,Total,Weekend,1.9,500
5,Other/Prefer not to say,Educational,Weekday,0.52,500
5,Other/Prefer not to say,Recreational,Weekday,0.98,500
5,Other/Prefer not to say,Total,Weekday,1.5,500
5,Other/Prefer not to say,Educational,Weekend,0.44,500
5,Other/Prefer not to say,Recreational,Weekend,1.51,500
5,Other/Prefer not to say,Total,Weekend,1.95,500
6,Male,Educational,Weekday,0.64,480
6,Male,Recreational

In [21]:
df = pd.read_csv(path)
df.head()

,Age,Gender,Screen Time Type,Day Type,Average Screen Time (hours),Sample Size
0,5,Male,Educational,Weekday,0.44,500
1,5,Male,Recreational,Weekday,1.11,500
2,5,Male,Total,Weekday,1.55,500
3,5,Male,Educational,Weekend,0.50,500
4,5,Male,Recreational,Weekend,1.44,500


In [34]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Load your dataset
df = pd.read_csv(path)

# Prepare subplots layout
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        "📈 Total Screen Time by Age",
        "🧩 Recreational vs Educational",
        "📊 Weekday vs Weekend",
        "🎮 Recreational Screen Time by Gender",
        "👨‍👩‍👧‍👦 Sample Size by Age"
    ),
    specs=[[{"type": "scatter"}, {"type": "domain"}],
           [{"type": "bar"}, {"type": "box"}],
           [{"type": "bar"}, None]],
    vertical_spacing=0.15
)

# 1. Line Plot: Total Screen Time by Age
total_df = df[df['Screen Time Type'] == 'Total']
for gender in total_df['Gender'].unique():
    for day in ['Weekday', 'Weekend']:
        subset = total_df[(total_df['Gender'] == gender) & (total_df['Day Type'] == day)]
        fig.add_trace(
            go.Scatter(x=subset['Age'], y=subset['Average Screen Time (hours)'],
                       mode='lines+markers',
                       name=f"{gender} - {day}"),
            row=1, col=1
        )

# 2. Pie Chart: Recreational vs Educational
pie_df = df[df['Screen Time Type'].isin(['Educational', 'Recreational'])].groupby(
    'Screen Time Type')['Average Screen Time (hours)'].mean().reset_index()
fig.add_trace(
    go.Pie(labels=pie_df['Screen Time Type'], values=pie_df['Average Screen Time (hours)']),
    row=1, col=2
)

# 3. Bar Chart: Weekday vs Weekend
grouped = df[df['Screen Time Type'] == 'Total'].groupby(['Age', 'Day Type'])['Average Screen Time (hours)'].mean().reset_index()
for day_type in grouped['Day Type'].unique():
    bar_data = grouped[grouped['Day Type'] == day_type]
    fig.add_trace(
        go.Bar(x=bar_data['Age'], y=bar_data['Average Screen Time (hours)'],
               name=day_type),
        row=2, col=1
    )

# 4. Box Plot: Recreational Screen Time by Gender
rec_df = df[df['Screen Time Type'] == 'Recreational']
for gender in rec_df['Gender'].unique():
    gender_data = rec_df[rec_df['Gender'] == gender]
    fig.add_trace(
        go.Box(y=gender_data['Average Screen Time (hours)'], name=gender),
        row=2, col=2
    )

# 5. Bar Chart: Sample Size by Age
sample_df = df[df['Screen Time Type'] == 'Total'].groupby('Age')['Sample Size'].mean().reset_index()
fig.add_trace(
    go.Bar(x=sample_df['Age'], y=sample_df['Sample Size'], name="Sample Size"),
    row=3, col=1
)

# Update layout
fig.update_layout(
    height=1000,
    width=1000,
    title_text="📊 Children's Screen Time Dashboard",
    showlegend=True
)

fig.show()
